In [3]:
import torch
import torch.nn.functional as F
from torch import nn
from torchvision import transforms
from torch.autograd import Variable
from ClassifierNew import ClassifierNew, Flatten

In [112]:
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]
input_size = (224, 224)
# test_transform = transforms.Compose([transforms.Resize(input_size), 
#                                     transforms.ToTensor(),
#                                     transforms.Normalize(norm_mean, norm_std)])
test_transform = transforms.Compose([transforms.Resize(input_size),
                                          transforms.RandomHorizontalFlip(),
                                          transforms.RandomVerticalFlip(),
                                          transforms.ToTensor(), 
                                          transforms.Normalize(norm_mean, norm_std)])

In [96]:
lesion_type_dict = {
    4 : 'Melanocytic nevi',
    6 : 'Vascular Lesions',
    2 : 'Benign keratosis-like lesions ',
    1 : 'Basal cell carcinoma',
    0 : 'Actinic keratoses',
    5 : 'Melanoma',
    3 : 'Dermatofibroma'
}

In [118]:
from PIL import Image

def predict(path, modelpath = "D:/Github/SkinCancerCapstone/models/resnet101-adam.pth"):
    model = torch.load(modelpath)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    
    img = Image.open(path)
    img = test_transform(img).float()
    img = Variable(img, requires_grad=False)
    img = img.unsqueeze(0).to(device)
    output = model(img)
    print(output)
    m = nn.Softmax(dim = 1)
    op = m(output)
    op = op.cpu().detach().numpy()[0]
    print(op)
    opind = op.argsort()[-3:][::-1]
    print(opind)
#     prediction = output.max(1, keepdim=True)[1].tolist()
    li =[]
    for p in opind:
        li.append((lesion_type_dict[p], op[p]))
    return li

In [130]:
predict("D:/Github/SkinCancerCapstone/report/img/skin_lesions/mel.jpg")
# predict("D:/Github/SkinCancerCapstone/data/HAM10000_images_part_1/ISIC_0027452.jpg")

tensor([[ 2.1213,  0.3717, -0.0059, -2.1351, -2.1645,  2.5128, -2.1774]],
       device='cuda:0', grad_fn=<AddmmBackward>)
[0.35540867 0.06178133 0.04235274 0.00503691 0.00489102 0.52570057
 0.00482876]
[5 0 1]


[('Melanoma', 0.52570057),
 ('Actinic keratoses', 0.35540867),
 ('Basal cell carcinoma', 0.06178133)]